In [1]:
import pandas as pd

In [2]:
alternatives = pd.read_csv("./data/milosz_alternatives1.csv")
criteria = pd.read_csv("./data/milosz_criteria1.csv")

In [3]:
alternatives

,country,power,safety,construction_cost
0,ITA,98,8,400
1,BEL,58,0,800
2,GER,66,5,1000
3,SWE,74,3,600
4,AUT,80,7,200
5,FRA,82,10,600


In [4]:
criteria

,criterion_name,criterion_type,indifference,preference,veto,weight
0,power,gain,0,0.000001,NaN,3
1,safety,gain,0,2.000000,NaN,2
2,construction_cost,cost,100,300.000000,NaN,5


In [5]:
from validate_input import validate_input
validate_input(alternatives, criteria)

True

In [6]:
print(criteria.iloc[0])

criterion_name       power
criterion_type        gain
indifference             0
preference        0.000001
veto                   NaN
weight                   3
Name: 0, dtype: object


In [7]:
# compute preference between alternatives a and b
def marginal_preference_index(a: pd.Series, b: pd.Series, criterion: pd.Series) -> float:
    p = criterion["preference"]
    q = criterion["indifference"]
    criterion_type = criterion["criterion_type"]
    criterion_name = criterion["criterion_name"]
    a_val = a[criterion_name]
    b_val = b[criterion_name]
    difference = a_val - b_val
    if criterion_type == "cost":
        difference *= -1

    if difference > p:
        return 1
    if difference <= q:
        return 0
    return (difference - q) / (p-q)

In [8]:
def comprehensive_preference_index(a: pd.Series, b: pd.Series, criteria: pd.DataFrame) -> float:
    numerator_sum = sum([marginal_preference_index(a, b, criterion[1])*criterion[1]["weight"] for criterion in criteria.iterrows()])
    sum_of_weights = sum([criterion[1]["weight"] for criterion in criteria.iterrows()])
    return numerator_sum/sum_of_weights

In [9]:
# TODO delete this cell after its outputs are discussed by the developers
# different results for FRA-AUT and AUT-FRA than in the presentation
# manual verification suggests that Milosz made an oopsie
for alternative in alternatives.iterrows():
    for b_alternative in alternatives.iterrows():
        if alternative[0] == b_alternative[0]:
            continue

        print(alternative[1][0], b_alternative[1][0], comprehensive_preference_index(alternative[1], b_alternative[1], criteria))

ITA BEL 1.0
ITA GER 1.0
ITA SWE 0.75
ITA AUT 0.4
ITA FRA 0.55
BEL ITA 0.0
BEL GER 0.25
BEL SWE 0.0
BEL AUT 0.0
BEL FRA 0.0
GER ITA 0.0
GER BEL 0.5
GER SWE 0.2
GER AUT 0.0
GER FRA 0.0
SWE ITA 0.0
SWE BEL 0.75
SWE GER 0.8
SWE AUT 0.0
SWE FRA 0.0
AUT ITA 0.25
AUT BEL 1.0
AUT GER 1.0
AUT SWE 1.0
AUT FRA 0.5
FRA ITA 0.2
FRA BEL 0.75
FRA GER 1.0
FRA SWE 0.5
FRA AUT 0.5


In [10]:
def positive_flow(alternative: pd.Series, alternatives: pd.DataFrame, criteria: pd.DataFrame) -> float:
    flow = 0
    for b_alternative in alternatives.iterrows():
        if alternative[0] == b_alternative[0]:
            continue
        flow += comprehensive_preference_index(alternative[1], b_alternative[1], criteria)
    return flow

def negative_flow(alternative, alternatives, criteria):
    flow = 0
    for b_alternative in alternatives.iterrows():
        if alternative[0] == b_alternative[0]:
            continue
        flow += comprehensive_preference_index(b_alternative[1], alternative[1], criteria)
    return flow

In [11]:
# again, the results for AUT and FRA are different than in the lecture
# they are however consistent with the results obtained earlier on in the notebook
for alternative in alternatives.iterrows():
    print(alternative[1][0],
          positive_flow(alternative, alternatives, criteria),
          negative_flow(alternative, alternatives, criteria))

ITA 3.7 0.45
BEL 0.25 4.0
GER 0.7 4.05
SWE 1.55 2.45
AUT 3.75 0.9
FRA 2.95 1.05


In [58]:
# TODO: implement
# still work in progress
def PROMETHEE1_ranking(alternatives: pd.DataFrame, criteria: pd.DataFrame):
    positive_flow_ranking = sorted([(a[1][0], positive_flow(a, alternatives, criteria)) 
                                     for a in alternatives.iterrows()],
                                     key = lambda x: x[1], reverse=True)
    negative_flow_ranking = sorted([(a[1][0], negative_flow(a, alternatives, criteria))
                                     for a in alternatives.iterrows()],
                                     key = lambda x: x[1])
    print(positive_flow_ranking)
    print(negative_flow_ranking)
    positive_flow_dict = dict((a[1][0], positive_flow(a, alternatives, criteria))
                               for a in alternatives.iterrows())
    negative_flow_dict = dict((a[1][0], negative_flow(a, alternatives, criteria))
                               for a in alternatives.iterrows())
    alternative_names = alternatives.iloc[:, 0].tolist()
    relations_dict = dict()
    for name in alternative_names:
        for different_name in alternative_names:
            if name == different_name:
                continue
            if ((positive_flow_dict[name] >= positive_flow_dict[different_name] and
                negative_flow_dict[name] < negative_flow_dict[different_name]) or
               (positive_flow_dict[name] > positive_flow_dict[different_name] and
                negative_flow_dict[name] <= negative_flow_dict[different_name])):
                relations_dict[(name, different_name)] = 'P'
            elif (positive_flow_dict[name] == positive_flow_dict[different_name] and
                  negative_flow_dict[name] == negative_flow_dict[different_name]):
                relations_dict[((name, different_name))] = 'I'
            elif ((positive_flow_dict[name] > positive_flow_dict[different_name] and
                negative_flow_dict[name] > negative_flow_dict[different_name]) or
               (positive_flow_dict[name] < positive_flow_dict[different_name] and
                negative_flow_dict[name] < negative_flow_dict[different_name])):
                relations_dict[(name, different_name)] = '?'
    # RELATIONS are key, no getting around that (ranking is not enough - indifference)
    # a ROOT of the ranking could be useful - an imaginary alternative, better than all others
    # (consider an example when the two rankings are just shifted)
    # maybe just solve all pairs LONGEST paths, where paths are created in the direction of preference
    
    # for k, v in relations_dict.items():
    #     print(k, v)

    # "FUN" fact - dict.fromkeys(alternatvie_names, []) makes all keys "point" to the same list
    preference_losers_dict = dict((alternative_name, []) for alternative_name in alternative_names)
    unranked_alternatives = alternative_names
    for pair, relation in relations_dict.items():
        if relation != "P":
            continue
        preference_losers_dict[pair[1]].append(pair[0])
    print(preference_losers_dict)
    ranking = []
    while unranked_alternatives:
        alternatives_to_add_to_ranking = []
        print("Unranked", unranked_alternatives)
        print("Dict:", preference_losers_dict)
        for alternative_name in unranked_alternatives:
            print(alternative_name)
            if preference_losers_dict[alternative_name]:
                continue
            alternatives_to_add_to_ranking.append(alternative_name)
            unranked_alternatives.remove(alternative_name)
            for unranked_alternative_name in unranked_alternatives:
                if alternative_name in preference_losers_dict[unranked_alternative_name]:
                    preference_losers_dict[unranked_alternative_name].remove(alternative_name)
        print(f"Appending: {alternatives_to_add_to_ranking}")
        ranking.append(alternatives_to_add_to_ranking)

    return ranking

In [59]:
PROMETHEE1_ranking(alternatives, criteria)

[('AUT', 3.75), ('ITA', 3.7), ('FRA', 2.95), ('SWE', 1.55), ('GER', 0.7), ('BEL', 0.25)]
[('ITA', 0.45), ('AUT', 0.9), ('FRA', 1.05), ('SWE', 2.45), ('BEL', 4.0), ('GER', 4.05)]
{'ITA': [], 'BEL': ['ITA', 'SWE', 'AUT', 'FRA'], 'GER': ['ITA', 'SWE', 'AUT', 'FRA'], 'SWE': ['ITA', 'AUT', 'FRA'], 'AUT': [], 'FRA': ['ITA', 'AUT']}
Unranked ['ITA', 'BEL', 'GER', 'SWE', 'AUT', 'FRA']
Dict: {'ITA': [], 'BEL': ['ITA', 'SWE', 'AUT', 'FRA'], 'GER': ['ITA', 'SWE', 'AUT', 'FRA'], 'SWE': ['ITA', 'AUT', 'FRA'], 'AUT': [], 'FRA': ['ITA', 'AUT']}
ITA
GER
SWE
AUT
Appending: ['ITA', 'AUT']
Unranked ['BEL', 'GER', 'SWE', 'FRA']
Dict: {'ITA': [], 'BEL': ['SWE', 'FRA'], 'GER': ['SWE', 'FRA'], 'SWE': ['FRA'], 'AUT': [], 'FRA': []}
BEL
GER
SWE
FRA
Appending: ['FRA']
Unranked ['BEL', 'GER', 'SWE']
Dict: {'ITA': [], 'BEL': ['SWE'], 'GER': ['SWE'], 'SWE': [], 'AUT': [], 'FRA': []}
BEL
GER
SWE
Appending: ['SWE']
Unranked ['BEL', 'GER']
Dict: {'ITA': [], 'BEL': [], 'GER': [], 'SWE': [], 'AUT': [], 'FRA': []}
BEL
A

[['ITA', 'AUT'], ['FRA'], ['SWE'], ['BEL'], ['GER']]

In [14]:
def PROMETHEE2_ranking(alternatives: pd.DataFrame, criteria: pd.DataFrame) -> list:
    ranking = sorted([(a[1][0], 
                       positive_flow(a, alternatives, criteria)-negative_flow(a, alternatives, criteria))
                       for a in alternatives.iterrows()], 
                       key = lambda x: x[1], reverse=True)
    ranking = [(name, round(score, 2)) for (name, score) in ranking]
    return ranking

In [15]:
# lecture error propagates as expected
PROMETHEE2_ranking(alternatives, criteria)

[('ITA', 3.25),
 ('AUT', 2.85),
 ('FRA', 1.9),
 ('SWE', -0.9),
 ('GER', -3.35),
 ('BEL', -3.75)]